# Compute results for the LiRA attack (Table 18)

In [1]:
from collections import defaultdict
import numpy as np
import os
import pickle
import scipy.stats

In [2]:
attacker_accesses = [
    # Shadow models trained using disjoint data and different seed w.r.t. target model.
    "shadow_dataset", 
    # Shadow models trained using disjoint data and same seed as the target model.
    "shadow_dataset_model_init"] 
datasets = ['cifar10', 'purchase100', 'texas100']
experiments_dirs = ['../experiments/cifar10/attack/cnn-large/lira',
                   '../experiments/purchase100/attack/generic-mlp-dropout_600,512,256,128,100/lira',
                   '../experiments/texas100/attack/generic-mlp-dropout_6169,1024,512,256,128,100/lira']
num_repetitions = 10

In [3]:
roc_results = { dataset: {attacker_access: defaultdict(list) for attacker_access in attacker_accesses}
               for dataset in datasets}

In [4]:
for dataset, experiments_dir in zip(datasets, experiments_dirs):
    for attacker_access in attacker_accesses:
        num_repetitions_found = 0
        for exp in range(num_repetitions):
            results_path = os.path.join(experiments_dir, attacker_access, f'results-{exp}.pickle')
            if not os.path.exists(results_path):
                continue
            with open(results_path, 'rb') as f:
                results = pickle.load(f)
            test_acc = results['test_acc']
            test_auc = results['test_auc']
            roc_results[dataset][attacker_access]['test_acc'].append(results['test_acc'])
            roc_results[dataset][attacker_access]['test_auc'].append(results['test_auc'])
            num_repetitions_found += 1
        mean_test_auc, std_test_auc = np.mean(roc_results[dataset][attacker_access]['test_auc']), np.std(roc_results[dataset][attacker_access]['test_auc'])
        mean_test_acc, std_test_acc = np.mean(roc_results[dataset][attacker_access]['test_acc']), np.std(roc_results[dataset][attacker_access]['test_acc'])
        h_test_auc =  std_test_auc * scipy.stats.t.ppf((1 + 0.95) / 2., num_repetitions_found-1) / (num_repetitions_found**0.5)
        print(f'Dataset {dataset} and {attacker_access}: {num_repetitions_found} experiments. ',  
              f'Test auc {mean_test_auc:.3f} ({h_test_auc:.3f}), test acc: {mean_test_acc:.1%} ({std_test_acc:.1%})')

Dataset cifar10 and shadow_dataset: 10 experiments.  Test auc 0.803 (0.018), test acc: 69.0% (1.0%)
Dataset cifar10 and shadow_dataset_model_init: 10 experiments.  Test auc 0.815 (0.017), test acc: 69.6% (0.9%)
Dataset purchase100 and shadow_dataset: 10 experiments.  Test auc 0.731 (0.015), test acc: 65.3% (0.9%)
Dataset purchase100 and shadow_dataset_model_init: 10 experiments.  Test auc 0.734 (0.016), test acc: 65.4% (0.9%)
Dataset texas100 and shadow_dataset: 10 experiments.  Test auc 0.829 (0.055), test acc: 68.9% (4.4%)
Dataset texas100 and shadow_dataset_model_init: 10 experiments.  Test auc 0.835 (0.059), test acc: 69.6% (4.8%)
